In [3]:
import jieba
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm

# 定义超参数
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 100
batch_size = 50
seq_length = 30
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}  # 单词到索引的映射
        self.idx2word = {}  # 索引到单词的映射
        self.idx = 0  # 索引计数器

    def __len__(self):
        return len(self.word2idx)

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx  # 将单词映射到索引
            self.idx2word[self.idx] = word  # 将索引映射到单词
            self.idx += 1  # 索引计数器加一
            
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        tokens = []  # 存储所有的单词
        lines = []  # 存储所有的行
        stopwords = [' ','wwwcrcom','www','cr173','com','□','=','txt','\n','本书来自www.cr173.com免费txt小说下载站','更多更新免费电子书请关注www.cr173.com','\u3000','目录']  # 定义广告
        with open(path, 'r', encoding="ANSI") as f:
            lines = f.readlines()  # 读取文件中的所有行
        for line in lines:
            for i in stopwords:
                line = line.replace(i,'')#去除停用词,广告
            words = jieba.lcut(line) + ['<eos>']  # 使用结巴分词对每一行进行分词，并在末尾添加结束标记
            tokens.extend(words)  # 将分词结果添加到tokens列表中
            for word in words:
                self.dictionary.add_word(word)  # 将分词结果添加到词典中

        ids = torch.LongTensor([self.dictionary.word2idx[word] for word in tokens])  # 将单词转换为对应的索引
        num_batches = len(ids) // batch_size
        ids = ids[:num_batches * batch_size].view(batch_size, -1)  # 调整ids的形状为(batch_size, -1)

        return ids  

# 定义LSTM模型类
class LSTMmodel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMmodel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        out = self.linear(out)

        return out, (h, c)

# 创建语料库实例并获取数据
corpus = Corpus()
ids = corpus.get_data('F:\\大学\\圆桌面\\深度学习与自然语言处理\\第三次作业\\jyxstxtqj_downcc.com\\鸳鸯刀.txt', batch_size)

# 获取词汇表大小
vocab_size = len(corpus.dictionary)

# 创建模型实例并将其移动到指定设备上
model = LSTMmodel(vocab_size, embed_size, hidden_size, num_layers).to(device)

# 定义损失函数和优化器
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
progress_bar = tqdm(total=len(range(0, ids.size(1) - seq_length, seq_length)) * num_epochs, desc='Training')

for epoch in range(num_epochs):
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))

    for i in range(0, ids.size(1) - seq_length, seq_length):
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        states = [state.detach() for state in states]
        outputs, states = model(inputs, states)
        loss = cost(outputs, targets.reshape(-1))

        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        progress_bar.update(1)  # 手动更新进度条位置

progress_bar.close()  # 关闭进度条
# 描述训练过程
print("训练完成！模型已经学习了语料库中的模式和规律。")
# 生成文本
num_samples = 1000
article = ""

state = (torch.zeros(num_layers, 1, hidden_size).to(device),
        torch.zeros(num_layers, 1, hidden_size).to(device))

prob = torch.ones(vocab_size)
_input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

for i in range(num_samples):
    output, state = model(_input, state)

    prob = output.exp()
    word_id = torch.multinomial(prob, num_samples=1).item()

    _input.fill_(word_id)

    word = corpus.dictionary.idx2word[word_id]
    word = '\n' if word == '<eos>' else word
    article += word
    
# 分词处理
word_list = jieba.lcut(article)
unique_words = set(word_list)  # 获取唯一单词
num_unique_words = len(unique_words)  # 统计唯一单词的数量
word_ratio = num_unique_words / num_samples  # 计算唯一单词的比例

print("词汇多样性评估：")
print("唯一单词数量:", num_unique_words)
print("词汇丰富性比例:", word_ratio)

# 统计重复率
repeated_phrases = 0
total_phrases = len(word_list) - 1

for i in range(total_phrases):
    if word_list[i] == word_list[i+1]:
        repeated_phrases += 1

repetition_ratio = repeated_phrases / total_phrases  # 计算重复率

print("重复率评估：")
print("重复词组数量:", repeated_phrases)
print("重复率:", repetition_ratio)
print(article)

Training: 100%|████████████████████████████████████████████████████████████████████| 1600/1600 [04:28<00:00,  5.95it/s]


训练完成！模型已经学习了语料库中的模式和规律。
词汇多样性评估：
唯一单词数量: 444
词汇丰富性比例: 0.444
重复率评估：
重复词组数量: 3
重复率: 0.0030120481927710845
捕来不再，人人心里又是难过，又是喜欢，道：「江湖上有言道：『做贼的心虚，放屁的脸红。』我可得定下神
，别让人瞧出了破绽。」
张镖师扬著单刀，从窗中跃出，左手一把却极出了出来。
萧半和心中虽然惊疑不定，却是丝毫不动声色，脸含微笑，说道：「村夫贱辰，敢
劳侍卫大人玉趾？」
卓天雄在京师中久闻萧半和的大名，但见他躯体雄伟，满腮虬髯，果然极是威武，
当下伸出右手，说道：「萧大侠千秋华诞，兄弟拜贺来迟，望乞恕罪。」萧半和笑道：
「好说，好说。」伸手与他相握。两人一运劲，手臂一震，均感半身酸嘛。这一下较量
，两人竟是功力悉敌，谁也不输於谁，当下携手同进寿堂。
两人之中，却是以卓天雄更加惊异，他以「震天三十掌」与「呼延十八鞭」称雄武
林，那「震天三十掌」唯有「混元气（请安官兵与
项颈之中，八方神骏满。卓天雄伸指在短刀上一弹，铮的利刃，出去动来。
众人正觉好笑，忽听得林外守望的一个弟子喝道：「是谁？」跟著另一人喝道：「
行！」萧中慧嗔道：「不错，不错！我不知道，我不知道。嘿嘿，鸳鸯刀啊鸳鸯刀！
」
那少女一听到「鸳鸯刀」三字，心中砰的一跳，将耳朵凑到墙壁上去，想听得仔细
些，但隔房刹时之间声息全无。那少女心中一动，从房门中溜了出去，悄步走到众镖师
的窗下一站。只听得周总镖师说道：「你怎知道？」袁冠南回过头来，说道：「却扇洞房，
便在一个『侠』字，定然非同小可
。江湖上有言道：『宁可不识字，不可不识人。』」於是抱拳说道：「久仰久仰！敝镖
局跟四侠素来没有过节，便请让道，日後专诚拜谒。」
盖一鸣双刺一击，叮叮作响，说道：「要让道那也不难，我们也不要你的镖银，只
须借一两件宝物用用，那也行了。」周威信道：「什麽宝物？」盖一鸣道：「嘿嘿，你
来问我，这可奇了。你自己不知道，我怎知道？」
周威信听到这里，知道今日之事决计不能善罢，这「太岳四侠」自是冲著自己背上
这对「鸳鸯刀」而来，心想：「江湖上有言道：『容情不动手，尚且不辞，何况区区几十两纹银？大哥、三弟、四弟，拿钱出来啊。我这
里有——」伸手到怀里一掏，单掌不开，原来衣囊中空空如也，连一文铜钱也没有。
幸好花剑影和盖一鸣身边都还有几两

In [43]:
import jieba
article="六粒盗，便问韦生道：“请你表演这项绝技。”又问左右道：“不用害怕，这件事与官人无关，不会累到你的。”说着提起皮囊，跃墙而出，体态轻盈，有一对夫妇。到后院，来到海边，戒备甚严，势似飞腾，寂无行迹，此必侠士，容貌。你性命，以匕首，大悦，收其策而退。当公之骋辩也，一妓有殊色，执红拂，立于前，独目公。公既去，而执拂者临轩，指吏曰：“问去者处士第几？，主人识得曰：“合负仆射万死。”这段文字既豪迈而又缠绵，有英雄之气，将他关了。王继恩说：“潘??公，主人西，便派你。”指着一名婢女道：“在下在市上有一间先父留下来的小店，每日可赚一贯，将复离叛。请郎君出，以金合授之。李郎明发，何日到太原？”靖计之曰：“主人西，则酒肆也。”公取到来，与姜廉夫同居如初。女剑仙水性杨花，男剑仙争风吃醋，都不成话。所以任渭长的评话说：“髑髅尽痴，剑仙如斯！”说着一名亲戚中，也知道了丈夫之意。那婢女当是她的心腹，她要丈夫一并杀了，以免道：“走罢！”董国庆又惊又喜，入房等妾同行。妾道：“我眼前有事，还不能走，明年"
# 分词处理
word_list = jieba.lcut(article)
unique_words = set(word_list)  # 获取唯一单词
num_unique_words = len(unique_words)  # 统计唯一单词的数量
word_ratio = num_unique_words / num_samples  # 计算唯一单词的比例

print("词汇多样性评估：")
print("唯一单词数量:", num_unique_words)
print("词汇丰富性比例:", word_ratio)

# 统计重复率
repeated_phrases = 0
total_phrases = len(word_list) - 1

for i in range(total_phrases):
    if word_list[i] == word_list[i+1]:
        repeated_phrases += 1

repetition_ratio = repeated_phrases / total_phrases  # 计算重复率

print("重复率评估：")
print("重复词组数量:", repeated_phrases)
print("重复率:", repetition_ratio)


词汇多样性评估：
唯一单词数量: 173
词汇丰富性比例: 0.173
重复率评估：
重复词组数量: 1
重复率: 0.0033444816053511705
